In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

In [3]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/anoop-vs/nlp-climate-change/refs/heads/main/main_data.csv"
)

In [4]:
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package stopwords to /Users/nafis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/nafis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/nafis/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [5]:
def preprocess(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    text = text.lower()
    words = text.split()
    stopword_list = set(stopwords.words("english"))
    words = [w for w in words if w not in stopword_list]
    stemmer = PorterStemmer()
    words = [stemmer.stem(w) for w in words]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(w) for w in words]
    text = " ".join(words)
    return text

In [6]:
df["content"] = df["content"].apply(preprocess)

In [7]:
from imblearn.over_sampling import RandomOverSampler

In [8]:
oversampler = RandomOverSampler(
    sampling_strategy={"Negative": 1842, "Positive": 1822, "Neutral": 1842}
)

In [9]:
X = df.drop("label", axis=1)
y = df["label"]

In [10]:
X_resampled, y_resampled = oversampler.fit_resample(X, y)

In [11]:
df_resampled = pd.concat([X_resampled, y_resampled], axis=1)

In [12]:
X = df_resampled["content"]

In [13]:
y = df_resampled["label"]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

### **TFIDF-COUNTVEC - LOGISTIC REGRESSION**


In [ ]:
tfidf_vectorizer = TfidfVectorizer()

In [16]:
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [17]:
count_vectorizer = CountVectorizer(max_features=5000)

In [18]:
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

In [ ]:
count_vectorizer = CountVectorizer()

In [20]:
clf = LogisticRegression()
clf.fit(X_train_combined, y_train)

/Users/nafis/code/python/climate change sentiment analysis/.venv311/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [21]:
y_pred = clf.predict(X_test_combined)

In [22]:
from sklearn.metrics import classification_report

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.89      0.98      0.93       389
     Neutral       0.80      0.71      0.75       370
    Positive       0.77      0.78      0.78       343

    accuracy                           0.83      1102
   macro avg       0.82      0.82      0.82      1102
weighted avg       0.82      0.83      0.82      1102



### TFIDF-COUNTVEC - RANDOM FOREST


In [23]:
rfctc = RandomForestClassifier()
rfctc.fit(X_train_combined, y_train)

RandomForestClassifier()

In [24]:
y_pred = rfctc.predict(X_test_combined)

In [25]:
from sklearn.metrics import classification_report

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.94      0.96      0.95       389
     Neutral       0.86      0.78      0.82       370
    Positive       0.81      0.87      0.84       343

    accuracy                           0.87      1102
   macro avg       0.87      0.87      0.87      1102
weighted avg       0.87      0.87      0.87      1102



### **TFIDF-COUNTVEC - SVM**


In [26]:
svmtc = SVC(C=10, gamma=10, kernel="linear")

In [27]:
svmtc.fit(X_train_combined, y_train)

SVC(C=10, gamma=10, kernel='linear')

In [28]:
y_pred = svmtc.predict(X_test_combined)

In [29]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.89      0.98      0.93       389
     Neutral       0.82      0.72      0.77       370
    Positive       0.75      0.77      0.76       343

    accuracy                           0.83      1102
   macro avg       0.82      0.82      0.82      1102
weighted avg       0.82      0.83      0.82      1102



### TFIDF-COUNTVEC - DECISION TREE


In [30]:
dttc = DecisionTreeClassifier(
    max_depth=None, max_features="auto", min_samples_leaf=1, min_samples_split=2
)

In [31]:
dttc.fit(X_train_combined, y_train)

/Users/nafis/code/python/climate change sentiment analysis/.venv311/lib/python3.11/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(


DecisionTreeClassifier(max_features='auto')

In [32]:
y_pred = dttc.predict(X_test_combined)

In [33]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.88      0.93      0.91       389
     Neutral       0.69      0.68      0.69       370
    Positive       0.70      0.67      0.69       343

    accuracy                           0.77      1102
   macro avg       0.76      0.76      0.76      1102
weighted avg       0.76      0.77      0.76      1102



### TFIDF-COUNTVEC - NAIVE BAYES


In [34]:
nbtc = MultinomialNB()

In [35]:
nbtc.fit(X_train_combined, y_train)

MultinomialNB()

In [36]:
y_pred = clf.predict(X_test_combined)

In [37]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.89      0.98      0.93       389
     Neutral       0.80      0.71      0.75       370
    Positive       0.77      0.78      0.78       343

    accuracy                           0.83      1102
   macro avg       0.82      0.82      0.82      1102
weighted avg       0.82      0.83      0.82      1102

